In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [4]:
base = pd.read_csv("stage3.csv")

In [5]:
pd.set_option('display.max_columns', None)
base.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [6]:
def convert_to_dict(value):
    if pd.isna(value):
        return value

    if '||' not in str(value):
        result_dict={0:value}
        return result_dict

    if '||' in str(value):
        pairs = value.split('||')
        pairs[1].strip("'])")

        result_dict = {}
        for pair in pairs:
            #Some are corrupted : 1: instead of ::
            if '::' in pair:
                key, val = pair.split('::', 1)
                result_dict[int(key)] = val
            else:
                key, val = pair.split(':', 1)
                result_dict[int(key)] = val
        return result_dict


updated_base = base.copy()
list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict)
updated_base.head()



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


Beaucoup des dictionnaires ne se sont pas bien convertis en raison de corruption de la base : certains sont codés avec '|' à la place de '||', nous essayons donc ici de gérer ces exceptions.

In [7]:

def convert_to_dict2(value2):
    if pd.isna(value2):
        return value2

    #some dict are corrupted : one | instead of ||
    list_value=list(value2.values())
    str_value= ''.join(list_value)

    if '|' not in str_value:
        return value2

    if '|' in str_value:
        
        pairs = str_value.split('|')
        pairs[1].strip("'])")

        result_dict = {}
        for pair in pairs:
            #Some are corrupted : 1: instead of ::
            if '::' in pair:
                key, val = pair.split('::', 1)
                result_dict[int(key)] = val
            elif ':' in pair:
                key, val = pair.split(':', 1)
                result_dict[int(key)] = val
            #because they already passed through convert_to_dict, the first key (0) is missing
            else:
                key, val=0,pair
                result_dict[key]=val
                print(val)
        return result_dict

list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict2)
updated_base.head()


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [8]:
def convert_to_list(value):
    if pd.isna(value):
        return value

    if '||' in str(value):
        liste = value.split('||')
        return liste


list_of_list_columns = ['incident_characteristics', 'sources']
updated_base[list_of_list_columns] = updated_base[list_of_list_columns].applymap(convert_to_list)
updated_base.head()



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0


### Début des statistiques descriptives 

import des packages nécessaires:

In [9]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)

In [10]:
import plotly.express as px

### Barplot décrivant le nombre d'incidents par arme à feu par Etat

In [11]:
fig = px.bar(updated_base, x=sorted(pd.unique(updated_base['state'])), y=updated_base.groupby('state').count()['incident_id'], title="Nombre d'incidents par arme à feu par Etat",
labels={"x":"Etat", "y":"nombre d'incidents"})

In [12]:
fig.show()

On remarque ici que les Etats dans lesquels le plus d'incidents par arme à feu ont lieu sont : l'Illinois (17.5k), la Californie (16.3k), la Floride (15k) et le Texas (13.5k). 
## A vérifier si cohérent avec la littérature sur le sujet 

### Graphique étudiant le nombre d'incidents par mois 

In [13]:
updated_base['date']=pd.to_datetime(updated_base['date'])

In [14]:
updated_base['month_year'] =updated_base['date'].dt.month.astype(str)+"-"+updated_base['date'].dt.year.astype(str)

In [15]:

fig2 = px.bar(updated_base, x=pd.unique(updated_base['month_year']), y=updated_base.groupby(updated_base['month_year']).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu par mois")


fig2

In [16]:

fig2bis = px.bar(updated_base, x=pd.unique(updated_base['date'].dt.year), y=updated_base.groupby(updated_base['date'].dt.year).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu par an")


fig2bis

On remarque que tous les 5 ou 6 mois très peu d'incidents sont renseignés : problème de récolte des données ? saisonnalité peu probable au vu de la variation des mois concernés.
De plus la base de données est incomplète en 2013 et en 2018, mais on observe tout de même une augmentation du nombre d'incidents sur les 4 années complètes de la base. 

### étude du nombre de morts et blessés par incident

In [17]:
fig3 = px.bar(updated_base, x=sorted(pd.unique(updated_base['n_killed'])), y=updated_base.groupby(updated_base['n_killed']).count()['incident_id'],
              labels={"x":"nombre de morts", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre d'incidents ayant entraîné des morts",
              log_y=True)

fig3

On voit ici que la grande majorité des incidents n'entraînent pas de mort (185.5k) et que près de 50.000 incidents provoquent une mort. Quelques outliers provoquent au delà de 10 morts, ils correspondent aux tueries dites "de masse" dont voici les références ci-dessous.

In [18]:
mass_kill=updated_base[updated_base['n_killed']>=10]
mass_kill

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year
194,480366,2013-09-16,District of Columbia,Washington Navy Yard,1336 Isaac Hull Ave,11,3,http://www.gunviolencearchive.org/incident/480366,https://www.washingtonpost.com/local/police-se...,False,1.0,"{0: 'Unknown', 1: 'Unknown'}","{0: '9mm', 1: 'Shotgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",38.8730,NAVSEA Building 197,-76.9977,2.0,"Male victim shot in the leg, first female vict...","{0: '59', 1: '53', 2: '51', 3: '53', 4: '62', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Female',...","{0: 'Michael Arnold', 1: 'Martin Bodrog', 2: '...",{0: '15::Co-worker'},"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.nbcwashington.com/news/local/Confi...,NaN,NaN,9-2013
92624,423223,2015-10-01,Oregon,Roseburg,1140 Umpqua College Rd,10,9,http://www.gunviolencearchive.org/incident/423223,http://www.kval.com/news/local/Nine-wounded-vi...,False,4.0,"{0: 'Unknown', 1: 'Unknown', 2: 'Unknown', 3: ...","{0: '223 Rem [AR-15]', 1: 'Handgun', 2: 'Handg...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",43.2628,Umpqua Community College,-123.2800,6.0,shots fired at Snyder Hall around 10:30 a.m. 7...,"{0: '19', 1: '18', 2: '59', 3: '18', 4: '33', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Male...","{0: 'Lucero Alcaraz', 1: 'Quinn Glen Cooper', ...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.statesmanjournal.com/story/news/20...,7.0,4.0,10-2015
101531,456893,2015-12-02,California,San Bernardino,1365 South Waterman Avenue,16,19,http://www.gunviolencearchive.org/incident/456893,http://www.latimes.com/local/lanow/la-me-ln-sa...,False,31.0,"{0: 'Not-stolen', 1: 'Not-stolen', 2: 'Not-sto...","{0: '223 Rem [AR-15]', 1: '223 Rem [AR-15]', 2...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",34.0758,Inland Regional Center,-117.2770,4.0,Second Location [police v shooter] 1700 block ...,"{0: '42', 1: '58', 2: '52', 3: '37', 4: '40', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Male', 5...","{0: 'Larry Daniel Kaufman', 1: 'Damian Meins',...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.sbsun.com/general-news/20151202/14...,40.0,23.0,12-2015
130448,577157,2016-06-12,Florida,Orlando,1912 S Orange Avenue,50,53,http://www.gunviolencearchive.org/incident/577157,https://www.reuters.com/article/us-orlando-sho...,False,5.0,"{0: 'Not-stolen', 1: 'Not-stolen'}","{0: '223 Rem [AR-15]', 1: '9mm'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",28.5195,Pulse,-81.3767,2.0,Names are coming in sporadically and every eff...,"{0: '34', 1: '23', 2: '20', 3: '22', 4: '36', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Edward Sotomayor Jr', 1: 'Stanley Almodov...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.cityoforlando.net/police/mass-shoo...,47.0,12.0,6-2016
217151,980577,2017-11-05,Texas,Sutherland Springs,216 4th St,27,20,http://www.gunviolencearchive.org/incident/980577,http://abc7.com/full-list-of-texas-church-shoo...,False,28.0,"{0: 'Unknown', 1: 'Unknown', 2: 'Unknown', 3: ...","{0: '223 Rem [AR-15]', 1: 'Rifle', 2: '9mm', 3...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",29.2733,First Ba

In [19]:
fig4 = px.bar(updated_base, x=sorted(pd.unique(updated_base['n_injured'])), y=updated_base.groupby(updated_base['n_injured']).count()['incident_id'],
              labels={"x":"nombre de blessés", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre de blessés par incident",
              log_y=True)

fig4

Le nombre de blessés par incidents suit globalement la même distribution que le nombre de morts par incident mais est plus étalée. Dans 142.5k cas il n'y a aucun blessé, mais dans 81.000 incidents il y a un blessé. Les incidents provoquant plus de 10 blessés sont rares, les voici dans un dataframe. On y retrouve les incidents très meurtriers (Orlando, Sutherland Springs) mais également d'autres n'ayant pas fait beaucoup de morts.

In [20]:
mass_injured=updated_base[updated_base['n_injured']>10]
mass_injured

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year
34,482771,2013-03-11,District of Columbia,Washington,1200 North Capitol Street,0,13,http://www.gunviolencearchive.org/incident/482771,https://www.washingtonpost.com/local/police-se...,False,1.0,"{0: 'Unknown', 1: 'Unknown', 2: 'Unknown', 3: ...","{0: '9mm', 1: '9mm', 2: '40 SW', 3: '40 SW'}","[Shot - Wounded/Injured, Drive-by (car to stre...",38.9062,Tyler House,-77.0099,4.0,Two .40 caliber weapons used may have been the...,"{0: '19', 1: '19', 2: '17', 15: '21'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Craig Steven Wilson', 1: 'Andrew Davon Al...",NaN,"{0: 'Unharmed, Arrested', 1: 'Unharmed, Arrest...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.washingtontimes.com/news/2013/may/...,NaN,NaN,3-2013
73,486209,2013-05-12,Louisiana,New Orleans,Frenchmen Street,0,19,http://www.gunviolencearchive.org/incident/486209,http://www.nola.com/crime/index.ssf/2015/09/mo...,False,2.0,{0: '0::Unknown'},{0: '0::7.62 [AK-47]'},"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",29.9861,NaN,-90.0595,1.0,ms; 19 inj. Rival gang members retaliation eff...,"{17: '10', 18: '10', 19: '19', 20: '23'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'JT', 19: 'Akein Scott', 20: 'Shawn Scott'...",NaN,"{0: 'Injured', 1: 'Injured', 2: 'Injured', 3: ...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.upi.com/Top_News/US/2013/05/12/Abo...,97.0,4.0,5-2013
200,480521,2013-09-19,Illinois,Chicago,1809 W 50th St,0,12,http://www.gunviolencearchive.org/incident/480521,http://www.nydailynews.com/news/national/chica...,False,3.0,{0: '0::Unknown'},{0: '0::7.62 [AK-47]'},"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",41.8030,Cornell Square Park,-87.6699,1.0,"Sources said the ""Brick Squad"" gang faction, a...","{0: '3', 1: '17', 2: '15', 3: '27', 4: '24', 5...","{0: 'Child 0-11', 1: 'Teen 12-17', 2: 'Teen 12...","{0: 'Male', 1: 'Female', 2: 'Male', 3: 'Male',...","{0: 'Deonta Howard', 13: 'Quinton M. Humphries...","{13: 'Gang vs Gang', 14: 'Gang vs Gang', 15: '...","{0: 'Injured', 1: 'Injured', 2: 'Injured', 3: ...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.chicagotribune.com/news/local/brea...,2.0,1.0,9-2013
213,480765,2013-10-05,California,Fresno,3883 E. Calwa #114,1,12,http://www.gunviolencearchive.org/incident/480765,http://sanfrancisco.cbslocal.com/2013/10/06/gu...,False,16.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.7054,Soul Brothers Clubhouse,-119.7600,NaN,NaN,{0: '0::18'},"{0: 'Adult 18+', 2: 'Adult 18+', 3: 'Adult 18+...",{0: '0::Male'},{0: '0::Dejuan Gladney'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",None,31.0,14.0,10-2013
239,493842,2013-11-09,Texas,Cypress,7300 block of Enchanted Creek Drive,2,16,http://www.gunviolencearchive.org/incident/493842,http://www.dailymail.co.uk/news/article-250059...,False,10.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",29.8840,NaN,-95.7187,NaN,Birthday party in home advertised on social me...,"{0: '17', 1: '16', 2: '16', 18: '21', 19: '18'}","{0: 'Teen 12-17', 1: 'Teen 12-17', 2: 'Teen 12...","{0: 'Male', 1: 'Female', 2: 'Male', 18: 'Male'...","{0: 'Queric Richardson', 1: 'Arielle Shepherd'...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.cbsnews.com/pictures/victims-of-th...,132.0,18.0,11-2013
6831,108626,2014-02-20,Florid

### Statistiques descriptives sur les suspects et auteurs des incidents 

Définition d'une fonction qui permet de récupérer les labels récurrents présents dans les dictionnaires.

In [21]:
def labels_uniques(nom_var):
    liste=[]
    for i in updated_base[nom_var]:
        if type(i)!=dict:
            continue
        else:
            liste.append(i.values())

    flat_list = [item for sublist in liste for item in sublist] 
    print(nom_var, pd.unique(flat_list))

variables_dict=['participant_status','participant_type','participant_age_group','participant_age',
    'participant_gender','gun_type', 'gun_stolen','participant_relationship']

for nom in variables_dict:
    labels_uniques(nom)

participant_status ['Arrested' 'Injured' 'Killed' 'Injured, Unharmed, Arrested'
 'Unharmed, Arrested' 'Unharmed' 'Injured, Arrested'
 'Killed, Unharmed, Arrested' 'Injured, Unharmed' 'Killed, Injured'
 '0::Injured' '0::Killed' '0::Unharmed' '0::Unharmed, Arrested'
 '1::Unharmed' '1::Killed' 'Killed, Unharmed' '0::Injured, Arrested'
 'Killed, Arrested' '1::Injured' '1::Unharmed, Arrested' '0:Injured'
 '0:Killed' '0:Unharmed' '1:Unharmed' '0:Unharmed, Arrested'
 '1:Unharmed, Arrested' '1:Killed' '0:Arrested' '1:Injured' '2::Unharmed'
 '0::Arrested' '2::Killed' '4::Unharmed, Arrested' '2::Arrested'
 '3::Unharmed' '4::Unharmed' '0::Killed, Unharmed' '1::Injured, Arrested'
 '2::Unharmed, Arrested' '0::Killed, Arrested' '2::Injured' '1::Arrested'
 '0::Injured, Unharmed' '0::Injured, Unharmed, Arrested'
 '0::Killed, Unharmed, Arrested' '3::Unharmed, Arrested'
 '2::Injured, Arrested' '3::Injured' '1::Killed, Unharmed'
 '0::Killed, Injured']
participant_type ['Victim' 'Subject-Suspect' '0::Subj

Les dictionnaires ne sont pas parfaits au moment où je commence ces statistiques au vu des valeurs uniques mais environ 15 erreurs sur 239 000 observations me paraît tout à fait raisonnable.
## attention c'est pas tout à fait ça !!

In [47]:
data_suspects=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Subject-Suspect':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_sus']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_sus']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_sus']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['genre_sus']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['genre_sus']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_sus']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_sus']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan
                
                if type(updated_base['participant_relationship'][index])==dict:
                    if key in updated_base['participant_relationship'][index].keys():
                        result_dict['relation']=updated_base['participant_relationship'][index][key]
                else:
                    result_dict['relation']=np.nan


                data_suspects.append(result_dict)

    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Subject-Suspect':
            result_dict={'id':index}

            if type(updated_base['participant_age_group'][index])==dict:
                if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_sus']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_sus']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_sus']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['genre_sus']=updated_base['participant_gender'][index][key]
            else:
                result_dict['genre_sus']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_sus']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_sus']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan

            if type(updated_base['participant_relationship'][index])==dict:
                if key in updated_base['participant_relationship'][index].keys():
                    result_dict['relation']=updated_base['participant_relationship'][index][key]
            else:
                result_dict['relation']=np.nan
                
            data_suspects.append(result_dict)

    index+=1


df_suspects=pd.DataFrame(data_suspects)

In [23]:
df_suspects.head()

,id,age_group_sus,genre_sus,status_sus,gun_type,gun_stolen,age_sus
0,0,Adult 18+,Female,Injured,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Adult 18+,Male,"Injured, Unharmed, Arrested",Unknown,Unknown,25
3,2,Adult 18+,Male,"Unharmed, Arrested",Unknown,Unknown,31
4,3,Adult 18+,Male,Killed,NaN,NaN,33


On obtient ici un dataframe permettant d'obtenir l'âge, le genre et le status de tous les participants aux incidents caractérisés comme 'Subject-Suspect' afin de pouvoir faire des statistiques descriptives. La colonne 'id' corresponds à l'index de l'incident concerné dans la base updated_base.

In [24]:
#il reste environ 10 erreurs de label concernant l'âge, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
groupes=['Adult 18+', 'Teen 12-17','Child 0-11']
filtered_suspects_age=df_suspects[df_suspects['age_group_sus'].isin(groupes)]

fig5 = px.bar(filtered_suspects_age, x=sorted(pd.unique(filtered_suspects_age['age_group_sus'])), y=filtered_suspects_age.groupby('age_group_sus').count()['id'],
              labels={"x":"âge du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par tranche d'âge")

fig5

In [25]:
age=list(range(10,90))
age=[str(num) for num in age]

filtered_suspects_age2=df_suspects[df_suspects['age_sus'].isin(age)]
filtered_suspects_age2

fig8 = px.bar(filtered_suspects_age2, x=sorted(pd.unique(filtered_suspects_age2['age_sus'])), y=filtered_suspects_age2.groupby('age_sus').count()['id'],
            labels={"x":"âge du suspect", "y":"nombre de suspects"},
            title="Nombre de suspect par tranche d'âge")

fig8

In [26]:
#il reste environ 10 erreurs de label concernant le genre, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
genre=['Male', 'Female']
filtered_suspects_genre=df_suspects[df_suspects['genre_sus'].isin(genre)]

fig6 = px.bar(filtered_suspects_genre, x=sorted(pd.unique(filtered_suspects_genre['genre_sus'])), y=filtered_suspects_genre.groupby('genre_sus').count()['id'],
              labels={"x":"genre du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par genre")

fig6

In [27]:
#il reste environ 10 erreurs de label concernant le statut, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
statut=['Injured','Injured, Unharmed, Arrested',
       'Unharmed, Arrested', 'Killed', 'Unharmed', 'Injured, Arrested',
       'Killed, Unharmed, Arrested', 'Arrested', 'Injured, Unharmed','Killed, Unharmed',
       'Killed, Arrested', 'Killed, Injured']
filtered_suspects_status=df_suspects[df_suspects['status_sus'].isin(statut)]

fig7 = px.bar(filtered_suspects_status, x=sorted(pd.unique(filtered_suspects_status['status_sus'])), y=filtered_suspects_status.groupby('status_sus').count()['id'],
              labels={"x":"statut du suspect", "y":"nombre de suspects (échelle logarithmique)"},
              title="Nombre de suspect par statut",
              log_y=True)

fig7

In [28]:
fig9 = px.bar(filtered_suspects_status, x=sorted(pd.unique(filtered_suspects_status['status_sus'])), y=filtered_suspects_status.groupby('status_sus').count()['id'],
              labels={"x":"statut du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par statut")

fig9

### Statistiques sur les caractéristiques des incidents 

In [29]:
liste_carac=[]
for i in updated_base["incident_characteristics"]:
    if type(i) != list:
        pass
    else:
        liste_carac.append(i)
flat_carac=[item for sublist in liste_carac for item in sublist]
pd.unique(flat_carac)

array(['Shot - Wounded/Injured',
       'Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)',
       'Possession (gun(s) found during commission of other crimes)',
       'Possession of gun by felon or prohibited person',
       'Shot - Dead (murder, accidental, suicide)', 'Gang involvement',
       'Shots Fired - No Injuries',
       'Bar/club incident - in or around establishment',
       'Officer Involved Incident',
       'Officer Involved Shooting - subject/suspect/perpetrator killed',
       'Drug involvement', 'Kidnapping/abductions/hostage',
       'Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator )',
       'Suicide^', 'Murder/Suicide',
       'Attempted Murder/Suicide (one variable unsuccessful)',
       'Domestic Violence', 'Home Invasion',
       'Home Invasion - Resident killed',
       'Armed robbery with injury/death and/or evidence of DGU found',
       'Drive-by (car to street, ca

On voit ici que ces données liées aux caractéristiques sont des champs libres non-normalisés donc on va d'abord s'intéresser aux descriptions comportant la mention de suicides car c'est, selon le Centers for Disease Control and Prevention's (CDC) National Center for Health Statistics, la cause principale de mort lors des incidents par arme à feu.

In [30]:
#création d'une indicatrice signalant la description d'un suicide

liste_indic=[]

for i in updated_base['incident_characteristics']:
    if i is not None:
        if "suicide" in str(i) :
            liste_indic.append(1)
        else:
            liste_indic.append(0)
    else:
        liste_indic.append(0)
    

updated_base['indic_suicide']=liste_indic
updated_base.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0,1-2013,1
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0,1-2013,1
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0,1-2013,1
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0,1-2013,1


In [31]:
fig10 = px.bar(updated_base[updated_base['n_killed']>=1], x=pd.unique(updated_base['indic_suicide']), y=updated_base[updated_base['n_killed']>=1].groupby(updated_base['indic_suicide']).count()['incident_id'],
              labels={"x":"incident impliquant un suicide (1) ou non (0)", "y":"nombre d'incidents"},
              title="Parmi les incidents impliquant au moins un mort, combien sont le résultat d'un suicide ?")

fig10

### Statistiques descriptives sur les victimes des incidents 

On commence par recréer une base similaire à celle des suspects pour récupérer les informations présentes dans les dictionnaires.

In [45]:
data_victimes=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Victim':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_vict']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_vict']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_vict']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['genre_vict']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['genre_vict']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_vict']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_vict']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan
            
                
                data_victimes.append(result_dict)


    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Victim':
            result_dict={'id':index}
            if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_vict']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_vict']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_vict']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['genre_vict']=updated_base['participant_gender'][index][key]
            else:
                result_dict['genre_vict']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_vict']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_vict']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan


            data_victimes.append(result_dict)

    index+=1


df_victim=pd.DataFrame(data_victimes)

In [33]:
df_victim.head()

,id,age_group_vict,age_vict,genre_vict,status_vict,gun_type,gun_stolen
0,0,Adult 18+,0::20,Male,Arrested,NaN,NaN
1,0,Adult 18+,NaN,Male,Injured,NaN,NaN
2,0,Adult 18+,NaN,NaN,Injured,NaN,NaN
3,0,Adult 18+,NaN,Male,Injured,NaN,NaN
4,1,Adult 18+,0::20,0::Male,Killed,NaN,NaN


On va faire des graphiques de statistiques descriptives similaires à ceux concernant les suspects:

In [34]:
#il reste environ 10 erreurs de label concernant l'âge, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
groupes=['Adult 18+', 'Teen 12-17','Child 0-11']
filtered_victim_age=df_victim[df_victim['age_group_vict'].isin(groupes)]

fig11 = px.bar(filtered_victim_age, x=sorted(pd.unique(filtered_victim_age['age_group_vict'])), y=filtered_victim_age.groupby('age_group_vict').count()['id'],
              labels={"x":"âge de la victime", "y":"nombre de victimes"},
              title="Nombre de victimes par tranche d'âge")

fig11

In [35]:
#il reste environ 10 erreurs de label concernant le genre, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
genre=['Male', 'Female']
filtered_victim_genre=df_victim[df_victim['genre_vict'].isin(genre)]

fig12 = px.bar(filtered_victim_genre, x=sorted(pd.unique(filtered_victim_genre['genre_vict'])), y=filtered_victim_genre.groupby('genre_vict').count()['id'],
              labels={"x":"genre de la victime", "y":"nombre de victimes"},
              title="Genre des victimes")

fig12

In [36]:
#il reste environ 10 erreurs de label concernant le statut, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
stat_vict=['Arrested', 'Injured', 'Killed', 'Unharmed',
       'Injured, Unharmed', 'Killed, Injured', 'Injured, Arrested', 'Unharmed, Arrested', 'Killed, Arrested',
       'Killed, Unharmed','Killed, Unharmed, Arrested', 'Injured, Unharmed, Arrested']
filtered_vict_status=df_victim[df_victim['status_vict'].isin(statut)]

fig13 = px.bar(filtered_vict_status, x=sorted(pd.unique(filtered_vict_status['status_vict'])), y=filtered_vict_status.groupby('status_vict').count()['id'],
              labels={"x":"statut de la victime", "y":"nombre de victimes"},
              title="Statut des victimes")

fig13

In [37]:
fig14 = px.bar(filtered_vict_status, x=sorted(pd.unique(filtered_vict_status['status_vict'])), y=filtered_vict_status.groupby('status_vict').count()['id'],
              labels={"x":"statut de la victime", "y":"nombre de victimes (échelle logarithmique)"},
              title="Statut des victimes", log_y=True)

fig14

In [38]:
age=list(range(100))
age=[str(num) for num in age]

filtered_vict_age2=df_victim[df_victim['age_vict'].isin(age)]
filtered_vict_age2

fig15 = px.bar(filtered_vict_age2, x=sorted(pd.unique(filtered_vict_age2['age_vict'])), y=filtered_vict_age2.groupby('age_vict').count()['id'],
            labels={"x":"âge de la victime", "y":"nombre de victimes"},
            title="Nombre de victimes par âge")

fig15

### Statistiques descriptives concernant les armes à feu utilisées.

In [39]:

list_guns=['Unknown', 'Handgun', '22 LR', '223 Rem [AR-15]',
    '9mm', '40 SW', 'Shotgun', '7.62 [AK-47]','45 Auto', '44 Mag','410 gauge', '32 Auto',
    'Other','Rifle','357 Mag', '38 Spl','25 Auto','12 gauge', '30-30 Win',
    '380 Auto','308 Win','20 gauge','16 gauge','30-06 Spr','300 Win', '10mm','28 gauge']

filtered_suspects_guns=df_suspects[df_suspects['gun_type'].isin(list_guns)]

fig16 = px.bar(filtered_suspects_guns, x=sorted(pd.unique(filtered_suspects_guns['gun_type'])), y=filtered_suspects_guns.groupby('gun_type').count()['id'],
              labels={"x":"arme utilisée", "y":"nombre de suspects concernés"},
              title="Nombre de suspect concernés par un incident avec ces armes")

fig16


In [40]:
fig17 = px.bar(filtered_suspects_guns, x=sorted(pd.unique(filtered_suspects_guns['gun_type'])), y=filtered_suspects_guns.groupby('gun_type').count()['id'],
              labels={"x":"arme utilisée", "y":"nombre de suspects concernés"},
              title="Nombre de suspect concernés par un incident avec ces armes",
              log_y=True)

fig17

### Tentative de mise en avant de corrélations sans base supplémentaire : cf stat_des_elo_corr

In [41]:
updated_base=updated_base.reset_index()
merge_sus=pd.merge(updated_base, df_suspects, how='outer', left_on='index', right_on='id')
merge_tot=pd.merge(merge_sus, df_victim, how='outer', right_on='id', left_on='index')
merge_tot.head()

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,id_x,age_group_sus,genre_sus,status_sus,gun_type_y,gun_stolen_y,age_sus,id_y,age_group_vict,age_vict,genre_vict,status_vict,gun_type,gun_stolen
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,0.0,Adult 18+,0::20,Male,Arrested,NaN,NaN
1,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN
2,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN
3,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN
4,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Inju

In [42]:
pd.unique(merge_tot['gun_type'])

array([nan, 'Handgun', '22 LR', '223 Rem [AR-15]', '0::Unknown',
       '0::Shotgun', '0::Handgun', '0::9mm', '0::45 Auto', '0::12 gauge',
       '0::7.62 [AK-47]', '40 SW', 'Shotgun', '7.62 [AK-47]', '9mm',
       'Unknown', '45 Auto', '0::44 Mag', '44 Mag', '0::Other',
       '0::40 SW', '0::38 Spl', '0::22 LR', '0::380 Auto', '410 gauge',
       '32 Auto', '0::308 Win', '0::357 Mag', 'Rifle', '357 Mag',
       '38 Spl', '0::30-30 Win', '30-30 Win', '0::Rifle',
       '0::223 Rem [AR-15]', '0:Unknown', '0:22 LR', '0:25 Auto', '0:9mm',
       '0:45 Auto', '0:Shotgun', '0:20 gauge', '0:357 Mag', '0:12 gauge',
       '0:40 SW', '0:Handgun', '0:44 Mag', '0:38 Spl', '0::32 Auto',
       '0::10mm', '0::20 gauge', 'Other', '0::25 Auto', '25 Auto',
       '12 gauge', '380 Auto', '308 Win', '0::28 gauge', '20 gauge',
       '0::300 Win', '0::410 gauge', '0::30-06 Spr', '0::16 gauge',
       '30-06 Spr', '10mm'], dtype=object)

In [43]:
list_guns=['Unknown', 'Handgun', '22 LR', '223 Rem [AR-15]',
    '9mm', '40 SW', 'Shotgun', '7.62 [AK-47]','45 Auto', '44 Mag','410 gauge', '32 Auto',
    'Other','Rifle','357 Mag', '38 Spl','25 Auto','12 gauge', '30-30 Win',
    '380 Auto','308 Win','20 gauge','16 gauge','30-06 Spr','300 Win', '10mm','28 gauge']

filtered_guns=merge_tot[merge_tot['gun_type'].isin(list_guns)]

fig18 = px.bar(merge_tot, x=sorted(pd.unique(filtered_guns['gun_type'])), y=filtered_guns[['gun_type', 'n_injured']].groupby('gun_type').mean()['n_injured'],
              labels={"x":"arme utilisée", "y":"nombre de blessés moyen"},
              title="Moyenne du nombre de blessés en fonction de l'arme utilisée")

fig18


A prendre avec grosses pincettes : database de 8000 incidents NON-UNIQUES (et dont le problème de non-unicité n'est ici pas géré donc certains évènements sont comptabilisés autant de fois qu'il y a de suspects).

### Statistiques descriptives concernant les relations de l'auteur des faits avec les victimes

La base de données filtrée ne comprenant que les suspects pour lesquels leur relation aux victimes a été bien renseignée comporte 7920 lignes, ce qui est très peu relativement à la base de données initiale mais reste assez important pour mettre en évidence le fait qu'une majorité des incidents pour lesquels cette donnée est renseignée sont des "Armed Robbery".

In [55]:
relations=['Family','Drive by - Random victims','Gang vs Gang','Armed Robbery','Home Invasion - Perp Does Not Know Victim',
    'Aquaintance', 'Significant others - current or former' ,'Home Invasion - Perp Knows Victim','Friends' ,
    'Mass shooting - Perp Knows Victims', 'Mass shooting - Random victims','Neighbor',
    'Co-worker']

filtered_suspects_relations=df_suspects[df_suspects['relation'].isin(relations)]

fig19 = px.bar(filtered_suspects_relations, x=sorted(pd.unique(filtered_suspects_relations['relation'])), y=filtered_suspects_relations.groupby('relation').count()['id'],
              labels={"x":"type de relation", "y":"nombre de relations concernés"},
              title="Nombre d'incidents dans lesquels ces relations/situations ont joué un rôle")

fig19

In [56]:
fig20 = px.bar(filtered_suspects_relations, x=sorted(pd.unique(filtered_suspects_relations['relation'])), y=filtered_suspects_relations.groupby('relation').count()['id'],
              labels={"x":"type de relation", "y":"nombre de relations concernées"},
              title="Nombre d'incidents dans lesquels ces relations/situations ont joué un rôle",
              log_y=True)

fig20

In [54]:
filtered_suspects_relations

,id,age_group_sus,genre_sus,status_sus,gun_type,gun_stolen,relation,age_sus
17,16,Adult 18+,Male,"Unharmed, Arrested",NaN,NaN,Drive by - Random victims,25
18,16,Adult 18+,Male,"Unharmed, Arrested",NaN,NaN,Drive by - Random victims,18
19,16,Adult 18+,Male,"Unharmed, Arrested",NaN,NaN,Drive by - Random victims,19
58,44,Adult 18+,Male,"Unharmed, Arrested",NaN,NaN,Gang vs Gang,19
59,44,Adult 18+,Male,"Unharmed, Arrested",NaN,NaN,Gang vs Gang,20
...,...,...,...,...,...,...,...,...
153750,239417,NaN,Male,Unharmed,NaN,NaN,Armed Robbery,NaN
153774,239478,Adult 18+,Male,"Unharmed, Arrested",NaN,NaN,Aquaintance,27
153775,239478,Adult 18+,Male,"Unharmed, Arrested",NaN,NaN,Aquaintance,29
153815,239555,Adult 18+,Male,Unharmed,NaN,NaN,Armed Robbery,NaN
